# <span style="color:#639DF3">Summarizer of CFA Research Challenge Presentations</span>

- This project creates an Python script able to summarize the CFA Research Challenge Presentations available on CFA Institute YouTube channel in an .md file.

In [18]:
import pytubefix # type: ignore
import ffmpeg # type: ignore
import openai # type: ignore
import sys # type: ignore
import os # type: ignore

In [19]:
# Accessing the OpeanAI API with your api_key
with open('openai_api.txt', 'r') as f:
    api_key = f.readline().strip()

client = openai.OpenAI(api_key=api_key)

In [ ]:
# Get the YouTube video using its URL
url = str(input('Insert the YouTube video URL: '))
filename = str(input('Insert the file name to be saved in your computer as .mp3: ')) + '.mp3'
yt = pytubefix.YouTube(url)
yt

In [ ]:
# Access the first stream of the video's audio
stream = yt.streams.filter(only_audio=True).first().url
stream

In [ ]:
# Process the stream with ffmpeg and save it as .mp3 file.
input_stream = ffmpeg.input(stream)
output_stream = ffmpeg.output(input_stream, filename, format="mp3", loglevel="error")
ffmpeg.run(output_stream)

In [23]:
# Generate the transcription using OpenAI Whisper
audio_file = open(filename, 'rb')
transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
).text

In [26]:
# Summarize the video's audio based on its transcription
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system",
         "content": """
         You are an assistant that summarize research challenge's presentations in detail. Answer with markdown format.
         """},
        {"role": "user",
         "content": f"Describe the following video: {transcript}"}
    ]
)

In [27]:
# Save the summary as .md file
summary_name = str(input('Insert the name of the summary file to be saved in your computer as .md:')) + '.md'
with open(summary_name, "w+") as md_file:
    md_file.write(completion.choices[0].message.content)